In [26]:
from dotenv import load_dotenv 
load_dotenv()

from fourlang.text_to_4lang import TextTo4lang
from fourlang.lexicon import Lexicon
from graphviz import Source
from scripts.parse_data import read_sherliic, build_graph
from scripts.similarity import Similarity

from tqdm import tqdm
preds = []
data = read_sherliic("data/test.csv", ud_path="data/relation_index.tsv", keep_context=True)
data_frame = build_graph(data)

In [27]:
data_frame["examples_A"] = data["examples_A"]
data_frame["examples_B"] = data["examples_B"]

In [28]:
data["prem_end"] = data["prem_end"].fillna('')
data["hypo_end"] = data["hypo_end"].fillna('')
data['premise_text'] = data["prem_argleft"] + " " + data["premise"].apply(lambda x: x.strip()) + " " + data["prem_argright"] + data["prem_end"]
data['hyp_text'] = data["hypo_argleft"] + " " + data["hypothesis"].apply(lambda x: x.strip()) + " " + data["hypo_argright"] + data["hypo_end"]

In [29]:
premise_texts = []
hyp_texts = []

for i, prem_text in enumerate(data["premise_text"]):
    example_A = data.iloc[i]["examples_A"].split("/")[0].strip()
    text = prem_text.replace("A", example_A)
    example_B = data.iloc[i]["examples_B"].split("/")[0].strip()
    text = text.replace("B", example_B)
    premise_texts.append(text)
    
for i, prem_text in enumerate(data["hyp_text"]):
    example_A = data.iloc[i]["examples_A"].split("/")[0].strip()
    text = prem_text.replace("A", example_A)
    example_B = data.iloc[i]["examples_B"].split("/")[0].strip()
    text = text.replace("B", example_B)
    hyp_texts.append(text)

In [30]:
data["premise_text"] = premise_texts
data["hyp_text"] = hyp_texts

In [31]:
import torch
from fairseq.data.data_utils import collate_tokens
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')
roberta.eval()

Using cache found in /home/adaamko/.cache/torch/hub/pytorch_fairseq_master


RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerSentenceEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layers): ModuleList(
          (0): TransformerSentenceEncoderLayer(
            (dropout_module): FairseqDropout()
            (activation_dropout_module): FairseqDropout()
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwi

In [7]:
batch_of_pairs = []

for i, text in enumerate(premise_texts):
    batch_of_pairs.append([text, hyp_texts[i]])

In [40]:
batch_of_pairs = [
    ["A is granting to B", "A is giving to B"],
    ["A is supporter of B", "A is backing B"],
    ["A is president of B","A is representing B"],
    ["A is interviewing B", "A is asking B"],
    ["A claims B", "A is wanting B"]
]

In [41]:
with torch.no_grad():

    batch = collate_tokens(
        [roberta.encode(pair[0], pair[1]) for pair in batch_of_pairs], pad_idx=1
    )

    logprobs = roberta.predict('mnli', batch)
    print(logprobs)
    guesses = logprobs.max(dim=1)
    # tensor([0, 2, 1, 0])

tensor([[-5.0970, -4.4486, -0.0180],
        [-5.6310, -4.6256, -0.0135],
        [-6.1340, -2.8906, -0.0594],
        [-3.4675, -3.2651, -0.0719],
        [-4.4199, -0.8968, -0.5446]])


In [42]:
import numpy as np

guesses_probs = logprobs.max(dim=1)
guesses = logprobs.argmax(dim=1)

In [43]:
guesses_probs = np.exp(guesses_probs[0].detach().numpy())

In [46]:
preds = []

for guess, prob in zip(guesses, guesses_probs):
    if guess == 2 and prob >= 0.8:
        preds.append(1)
    else:
        preds.append(0)

In [47]:
preds

[1, 1, 1, 1, 0]

In [12]:
gold = data_frame.score.tolist()

In [13]:
from sklearn.metrics import precision_recall_fscore_support as pr
#bPrecis, bRecall, bFscore, bSupport = pr(data_frame.score.tolist(), [1 if i>=1.0 else 0 for i in guesses])
bPrecis, bRecall, bFscore, bSupport = pr(gold, preds)

print("Precision: " +  str(bPrecis[1]))
print("Recall: " +  str(bRecall[1]))
print("Fscore: " +  str(bFscore[1]))
print(bSupport)

Precision: 0.7001845018450185
Recall: 0.7635814889336016
Fscore: 0.7305101058710299
[1995  994]


In [201]:
from sklearn.metrics import precision_recall_fscore_support as pr
#bPrecis, bRecall, bFscore, bSupport = pr(data_frame.score.tolist(), [1 if i>=1.0 else 0 for i in guesses])
bPrecis, bRecall, bFscore, bSupport = pr(gold, all_rules)

print("Precision: " +  str(bPrecis[1]))
print("Recall: " +  str(bRecall[1]))
print("Fscore: " +  str(bFscore[1]))
print(bSupport)

Precision: 0.3715651135005974
Recall: 0.6257545271629779
Fscore: 0.4662668665667167
[1995  994]


In [25]:
from sklearn.metrics import precision_recall_fscore_support as pr
#bPrecis, bRecall, bFscore, bSupport = pr(data_frame.score.tolist(), [1 if i>=1.0 else 0 for i in guesses])
bPrecis, bRecall, bFscore, bSupport = pr(gold, fourlang_preds)

print("Precision: " +  str(bPrecis[1]))
print("Recall: " +  str(bRecall[1]))
print("Fscore: " +  str(bFscore[1]))
print(bSupport)

Precision: 0.8640776699029126
Recall: 0.08953722334004025
Fscore: 0.16226071103008205
[1995  994]


In [24]:
fourlang_guesses = []

with open("fourlang_guesses_nofilt_nomulti_zero", "r+") as f:
    for line in f:
        fourlang_guesses.append(float(line.strip()))
        
fourlang_preds = [1 if i>=0.8 else 0 for i in fourlang_guesses]

In [199]:
all_rules = []

with open("results/wordnet+all_rules.csv", "r+") as f:
    for line in f:
        line = line.split()
        p = int(line[-1].strip())
        all_rules.append(p)
        
all_rules = all_rules[:2989]

In [203]:
combined_preds = []

for i, pred in enumerate(preds):
    if pred == 1 or fourlang_preds[i] == 1:
        combined_preds.append(1)
    else:
        combined_preds.append(0)

In [ ]:
new_positives = []

for i, pred in enumerate(combined_preds):
    if pred == 1 and gold[i] == 1 and preds[i] == 0:
        new_positives.append(i)
        
new_positives

In [ ]:
set(fourlang_true_pos) & set(alberta_false_neg)

In [ ]:
index = 5
print(premise_texts[false_negatives[index]])
print(hyp_texts[false_negatives[index]])

In [14]:
text_to_4lang = TextTo4lang(lang="en")

2020-12-18 14:45:09 INFO: Downloading default packages for language: en (English)...
INFO:stanza:Downloading default packages for language: en (English)...
2020-12-18 14:45:09 INFO: File exists: /home/adaamko/stanza_resources/en/default.zip.
INFO:stanza:File exists: /home/adaamko/stanza_resources/en/default.zip.
2020-12-18 14:45:13 INFO: Finished downloading models and saved to /home/adaamko/stanza_resources.
INFO:stanza:Finished downloading models and saved to /home/adaamko/stanza_resources.
2020-12-18 14:45:13 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| sentiment | sstplus   |
| ner       | ontonotes |

INFO:stanza:Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
|

In [15]:
import networkx as nx
from networkx import algorithms

import re
def clear_node(node):
    """
    Clears the node from the 4lang id parts
    :param node: the text to clear
    :return: the cleared text
    """
    node = re.sub(r'_[0-9][0-9]*', '', node)
    #if node in stopwords:
    #    node = "STOPWORD"
    return node

def whitelisting(graph, from_node):
    whitelist = []
    zero_graph = nx.MultiDiGraph()
    zero_graph.add_node(from_node)
    delete_list = []
    for edge in graph.G.edges(data=True):
        if not edge[2]["color"]:
            zero_graph.add_edge(edge[0], edge[1])
            
    for node in zero_graph.nodes():
        if algorithms.has_path(zero_graph, from_node, node):
            if node != from_node:
                whitelist.append(node)
    
    """
    for edge in zero_graph.edges(data=True):
        if edge[0] == from_node:
            whitelist.append(edge[1])
    """
    
    return whitelist

In [ ]:
graph_test = text_to_4lang.process_text("A is colonizing B", method="expand", depth=1, blacklist=["in", "on", "of"], filt=False, black_or_white="black", multi_definition=False, apply_from_depth=2)
print(graph_test.root)

In [ ]:
whitelisting(graph_test, graph_test.root)

In [16]:
from graphviz import Source
from tqdm import tqdm

In [17]:
preds = []

with torch.no_grad():
    for i, (prem, hyp) in enumerate(zip(premise_texts, hyp_texts)):
        #print(i)
        if guesses[i] == 2 and guesses_probs[i] >= 0.8:
            print("True found..")
            preds.append(1)
        else:
            batches = []
            
            text_to_process = data.iloc[i]["premise_text"]
            graph = text_to_4lang.process_text(text_to_process, method="expand", depth=1, blacklist=["in", "on", "of"], filt=False, black_or_white="black", multi_definition=False, apply_from_depth=2)

            if graph.root:
                reducing_nodes = whitelisting(graph, graph.root)
                graph_root = clear_node(graph.root)
                for node in reducing_nodes:
                    node = clear_node(node)
                    if node != "A" and node != "B":
                        
                        prem_reduced = prem
                        for word in prem_reduced.split():
                            if word.startswith(graph_root) and (node not in prem_reduced.split()):
                                prem_reduced = prem.replace(word, node)
                                batches.append([prem_reduced, hyp])
                label = 0
                if batches:
                    batch = collate_tokens(
                        [roberta.encode(pair[0], pair[1]) for pair in batches], pad_idx=1
                    )

                    logprobs_batch = roberta.predict('mnli', batch)
                    reduced_guesses = logprobs_batch.argmax(dim=1)
                    reduced_guesses_probs = logprobs_batch.max(dim=1)
                    reduced_guesses_probs = np.exp(reduced_guesses_probs[0].detach().numpy())

                    for guess, prob in zip(reduced_guesses, reduced_guesses_probs):
                        if guess == 2 and prob >= 0.95:
                            label = 1
                    
                    if label == 1 and gold[i] == 0:
                        print(batches)
                        print(text_to_process)
                        print(f"reduced: {reduced_guesses}")
                preds.append(label)

            else:
                preds.append(0)


[['Indiana Pacers is follow George W. Bush', 'George W. Bush is losing to Indiana Pacers'], ['Indiana Pacers is tail George W. Bush', 'George W. Bush is losing to Indiana Pacers'], ['Indiana Pacers is behind George W. Bush', 'George W. Bush is losing to Indiana Pacers']]
Indiana Pacers is trailing George W. Bush
reduced: tensor([2, 2, 0])
True found..


True found..


True found..
True found..


True found..
True found..


True found..
True found..


...

True found..


True found..


[['FOX is cross Futurama', 'Futurama is airing on FOX'], ['FOX is carnivore Futurama', 'Futurama is airing on FOX'], ['FOX is vulpe Futurama', 'Futurama is airing on FOX']]
FOX is cancelling Futurama
reduced: tensor([2, 2, 2])
True found..
True found..


True found..
True found..
[['Google is move against AFC North', 'Google is losing to AFC North'], ['Google is trademark against AFC North', 'Google is losing to AFC North']]
Google is going against AFC North
reduced: tensor([0, 2])
True found..
True found..
True found..
True found..
True found..


KeyboardInterrupt: 

## Semeval

In [215]:
from scripts.parse_data import read
import os
import pandas as pd

In [216]:
def read(lang1, lang2=None, graded=True):
    filename = "SemEval2020-Task2-Dev"
    language_name = ""
    if lang2 is None:
        filename = os.path.join(filename, "monolingual")
        language_name = lang1
    else:
        filename = os.path.join(filename, "cross-lingual")
        language_name = "-".join([lang1, lang2]) if lang1[0] < lang2[0] else "-".join([lang2, lang1])
    grad_or_bin = "graded" if graded else "binary"
    filename = os.path.join(filename, grad_or_bin, ".".join([language_name, grad_or_bin, "dev.data.txt"]))
    df = pd.read_csv(filename, delimiter=" ", header=None, names=["premise", "hypothesis", "score"])
    # if lang2 is not None:
    #     df.premise = df.premise.str.replace(lang1 + "_", "")
    #     df.premise = df.premise.str.replace(lang2 + "_", "")
    #     df.hypothesis = df.hypothesis.str.replace(lang1 + "_", "")
    #     df.hypothesis = df.hypothesis.str.replace(lang2 + "_", "")
    return df

In [217]:
data_frame = read("en", graded=False)

In [218]:
data_frame

,premise,hypothesis,score
0,custard,building,0
1,flashlight,agency,0
2,sandwich,food,1
3,enemy,friend,0
4,incense,aroma,1
...,...,...,...
311,selection,choice,1
312,material,rubber,0
313,beat,defeat,1
314,ape,beast,1


In [219]:
batch_of_pairs = []
premise_texts = data_frame["premise"].tolist()
hyp_texts = data_frame["hypothesis"].tolist()

for i, text in enumerate(premise_texts):
    batch_of_pairs.append([text, hyp_texts[i]])

In [ ]:
with torch.no_grad():

    batch = collate_tokens(
        [roberta.encode(pair[0], pair[1]) for pair in batch_of_pairs], pad_idx=1
    )

    logprobs = roberta.predict('mnli', batch)
    print(logprobs)
    guesses = logprobs.max(dim=1)

In [221]:
import numpy as np

guesses_probs = logprobs.max(dim=1)
guesses = logprobs.argmax(dim=1)

In [222]:
guesses_probs = np.exp(guesses_probs[0].detach().numpy())

In [229]:
preds = []

for guess, prob in zip(guesses, guesses_probs):
    if guess == 2:# and prob >= 0.8:
        preds.append(1)
    else:
        preds.append(0)

In [230]:
gold = data_frame.score.tolist()

In [231]:
from sklearn.metrics import precision_recall_fscore_support as pr
#bPrecis, bRecall, bFscore, bSupport = pr(data_frame.score.tolist(), [1 if i>=1.0 else 0 for i in guesses])
bPrecis, bRecall, bFscore, bSupport = pr(gold, preds)

print("Precision: " +  str(bPrecis[1]))
print("Recall: " +  str(bRecall[1]))
print("Fscore: " +  str(bFscore[1]))
print(bSupport)

Precision: 0.8325123152709359
Recall: 0.949438202247191
Fscore: 0.8871391076115485
[138 178]
